<a href="https://colab.research.google.com/github/besherh/DM-ML/blob/master/text_classification_bbc_news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Load libraries

In [58]:
import tensorflow as tf
import numpy as np
import tensorflow_datasets as tfds
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences



print(tf.__version__)
if(tf.test.is_gpu_available):
  print("GPU")
else:
  print("GPU is not available")

2.0.0-rc0
GPU


#Download BBC new dataset

In [4]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/bbc-text.csv \
    -O /tmp/bbc-text.csv

--2019-09-10 10:36:58--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/bbc-text.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.20.128, 2607:f8b0:400e:c07::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.20.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5057493 (4.8M) [application/octet-stream]
Saving to: ‘/tmp/bbc-text.csv’

/tmp/bbc-text.csv   100%[===================>]   4.82M  --.-KB/s    in 0.02s   

2019-09-10 10:36:59 (235 MB/s) - ‘/tmp/bbc-text.csv’ saved [5057493/5057493]



In [0]:
vocab_size = 1000
embedding_dim = 16
max_length = 120
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_portion = .8

In [56]:
sentences = []
labels = []
stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]
print(len(stopwords))
# Expected Output
# 153

153


In [57]:
with open("/tmp/bbc-text.csv", 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    for row in reader:
        labels.append(row[0])
        sentence = row[1]
        for word in stopwords:
            token = " " + word + " "
            sentence = sentence.replace(token," ")
        sentences.append(sentence)

print(len(labels))
print(len(sentences))
print(sentences[0])


2225
2225
tv future hands viewers home theatre systems  plasma high-definition tvs  digital video recorders moving living room  way people watch tv will radically different five years  time.  according expert panel gathered annual consumer electronics show las vegas discuss new technologies will impact one favourite pastimes. us leading trend  programmes content will delivered viewers via home networks  cable  satellite  telecoms companies  broadband service providers front rooms portable devices.  one talked-about technologies ces digital personal video recorders (dvr pvr). set-top boxes  like us s tivo uk s sky+ system  allow people record  store  play  pause forward wind tv programmes want.  essentially  technology allows much personalised tv. also built-in high-definition tv sets  big business japan us  slower take off europe lack high-definition programming. not can people forward wind adverts  can also forget abiding network channel schedules  putting together a-la-carte entertai

#Create training/Validation sets

In [88]:
train_size = int (len(sentences) * training_portion)
print(train_size)
train_sentences = sentences[:train_size]
train_labels = labels[:train_size]

val_sentences = sentences[train_size:]
val_labels = labels[train_size:]

print("training size", len(train_sentences))
print("validation size", len(val_sentences))

1780
training size 1780
validation size 445


#prepare the text corpus

In [0]:
tokenizer = Tokenizer(num_words= vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index

train_sequences =  tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_sequences, maxlen= max_length, padding=padding_type, truncating=trunc_type)

val_sequences = tokenizer.texts_to_sequences(val_sentences)
val_padded = pad_sequences(val_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

labels_tokenizer = Tokenizer()
labels_tokenizer.fit_on_texts(labels)
train_labels_sequence = np.array(labels_tokenizer.texts_to_sequences(train_labels))
val_labels_sequence = np.array(labels_tokenizer.texts_to_sequences(val_labels))




#Sequential model

In [118]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(6, activation='softmax')
])
model.summary()



Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_27 (Embedding)     (None, 120, 16)           16000     
_________________________________________________________________
global_average_pooling1d_8 ( (None, 16)                0         
_________________________________________________________________
dense_44 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_45 (Dense)             (None, 6)                 102       
Total params: 16,374
Trainable params: 16,374
Non-trainable params: 0
_________________________________________________________________


In [119]:
epoch = 30
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
history = model.fit(train_padded,train_labels_sequence, epochs= epoch, validation_data=(val_padded,val_labels_sequence))

Train on 1780 samples, validate on 445 samples
Epoch 1/30
1780/1780 [==============================] - 1s 412us/sample - loss: 1.7587 - accuracy: 0.2382 - val_loss: 1.7218 - val_accuracy: 0.3348
Epoch 2/30
1780/1780 [==============================] - 0s 104us/sample - loss: 1.6748 - accuracy: 0.4062 - val_loss: 1.6195 - val_accuracy: 0.3798
Epoch 3/30
1780/1780 [==============================] - 0s 107us/sample - loss: 1.5542 - accuracy: 0.4713 - val_loss: 1.4889 - val_accuracy: 0.5146
Epoch 4/30
1780/1780 [==============================] - 0s 100us/sample - loss: 1.4027 - accuracy: 0.5843 - val_loss: 1.3295 - val_accuracy: 0.5730
Epoch 5/30
1780/1780 [==============================] - 0s 114us/sample - loss: 1.2217 - accuracy: 0.6466 - val_loss: 1.1515 - val_accuracy: 0.6809
Epoch 6/30
1780/1780 [==============================] - 0s 104us/sample - loss: 1.0376 - accuracy: 0.7567 - val_loss: 0.9889 - val_accuracy: 0.7933
Epoch 7/30
1780/1780 [==============================] - 0s 109us/